### Write a program to implement the following services of PGP. You have to follow all the steps mentioned in the algorithms. a. Authentication b. Confidentiality for transmitting data.

In [1]:
!pip install pycryptodome

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.8 MB 465.5 kB/s eta 0:00:04
   -- ------------------------------------- 0.1/1.8 MB 595.3 kB/s eta 0:00:03
   ---- ----------------------------------- 0.2/1.8 MB 827.9 kB/s eta 0:00:02
   ------ --------------------------------- 0.3/1.8 MB 1.0 MB/s eta 0:00:02
   ----------- ---------------------------- 0.5/1.8 MB 1.6 MB/s eta 0:00:01
   ------------- -------------------------- 0.6/1.8 MB 1.6 MB/s eta 0:00:01
   ------------------- -------------------- 0.9/1.8 MB 2.2 MB/s eta 0:00:01
   --------------------------- ------------ 1.2/1.8 MB 2.7 MB/s eta 0:00:01
   ------------------------------------- -- 1.7/1.8 MB 3.2 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 3.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import zlib, secrets, base64
from Crypto.PublicKey import RSA
from Crypto.Hash import SHA1
from Crypto.Signature import pkcs1_15
from Crypto.Cipher import DES3, PKCS1_OAEP
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad

def b64(data, maxlen=50):
    s = base64.b64encode(data).decode()
    return s if len(s) <= maxlen else s[:maxlen] + "..."

# Keys
A_priv = RSA.generate(2048);  A_pub = A_priv.publickey()
B_priv = RSA.generate(2048);  B_pub = B_priv.publickey()

# Sender
M = b"Catch Me If You Can"
print(f"\n[M]   Plaintext message: {M.decode()}")

h = SHA1.new(M)
print(f"[H]   Compute SHA-1 over M: {b64(h.digest())}")

S = pkcs1_15.new(A_priv).sign(h)
print(f"[EP]  Sign H(M) with PR_a -> signature S: {b64(S)}")

signed_plain = len(S).to_bytes(2,"big") + S + M
print(f"[||]  Concatenate S and M (S || M): {b64(signed_plain)}")

compressed = zlib.compress(signed_plain, level=9)
print(f"[Z]   Compress (S || M): {b64(compressed)}")

Ks_raw = secrets.token_bytes(16)   # 128-bit
Ks = DES3.adjust_key_parity(Ks_raw)
pt = pad(compressed, 8)
C = DES3.new(Ks, DES3.MODE_ECB).encrypt(pt)
print(f"[EC]  Encrypt with 3DES-CFB (Ks): Ks={b64(Ks)} Ciphertext (C): {b64(C)}")

E_Ks = PKCS1_OAEP.new(B_pub).encrypt(Ks)
print(f"[EP]  Encrypt session key with PU_b -> E[PU_b, K_s]: {b64(E_Ks)}")

packet = (E_Ks, C)
print(f"[||]  Build transmitted packet: E[PU_b,K_s] || C")

#Receiver
Ks_recv = PKCS1_OAEP.new(B_priv).decrypt(packet[0])
print(f"\n[DP]   Decrypt E[PU_b,K_s] with PR_b -> recovered Ks: {b64(Ks_recv)}")

pt_recv = DES3.new(Ks_recv, DES3.MODE_ECB).decrypt(packet[1])
compressed_recv = unpad(pt_recv, 8)
print(f"[DC]   Decrypt C with Ks -> compressed data: {b64(compressed_recv)}")

signed_plain_recv = zlib.decompress(compressed_recv)
print(f"[Z^-1] Decompress -> (S || M): {b64(signed_plain_recv)}")

sig_len = int.from_bytes(signed_plain_recv[:2], "big")
S_recv = signed_plain_recv[2:2+sig_len]
M_recv = signed_plain_recv[2+sig_len:]
print(f"[M]    Extract plaintext M': {M_recv.decode()}")

h2 = SHA1.new(M_recv)
print(f"[DP]   Verify signature S with PU_a over SHA-1(M'): {b64(h2.digest())}")
try:
    pkcs1_15.new(A_pub).verify(h2, S_recv)
    verify_ok = True
except (ValueError, TypeError):
    verify_ok = False

print(f"[H]    Receiver recomputes SHA-1(M'): {b64(h2.digest())}")
print(f"[Compare] Signature/Hash check: {'VALID' if verify_ok else 'INVALID'}")


[M]   Plaintext message: Catch Me If You Can
[H]   Compute SHA-1 over M: HJIFjbK/iDgbO2K4SR0jajOY5K8=
[EP]  Sign H(M) with PR_a -> signature S: SwbDFwQBdoV2cpQF/ZdAYg5NRlQU6QyIMZpHJ/vWrLj3JCKh0q...
[||]  Concatenate S and M (S || M): AQBLBsMXBAF2hXZylAX9l0BiDk1GVBTpDIgxmkcn+9asuPckIq...
[Z]   Compress (S || M): eNoBFQHq/gEASwbDFwQBdoV2cpQF/ZdAYg5NRlQU6QyIMZpHJ/...
[EC]  Encrypt with 3DES-CFB (Ks): Ks=mLnIE1RY2taYSUrf6V0fUQ== Ciphertext (C): Aeq8DcFF34k9nqVsJrRt5+PQGOh8IFEGntF3WcwCDH3xTAGz3+...
[EP]  Encrypt session key with PU_b -> E[PU_b, K_s]: ohqYJ3PeHQyInT4RThZFqNnMWVuI690ak6FZ5W/gO9F+q6JzMT...
[||]  Build transmitted packet: E[PU_b,K_s] || C

[DP]   Decrypt E[PU_b,K_s] with PR_b -> recovered Ks: mLnIE1RY2taYSUrf6V0fUQ==
[DC]   Decrypt C with Ks -> compressed data: eNoBFQHq/gEASwbDFwQBdoV2cpQF/ZdAYg5NRlQU6QyIMZpHJ/...
[Z^-1] Decompress -> (S || M): AQBLBsMXBAF2hXZylAX9l0BiDk1GVBTpDIgxmkcn+9asuPckIq...
[M]    Extract plaintext M': Catch Me If You Can
[DP]   Verify signature S with 